#### Import libraries

In [52]:
import numpy as np
import pandas as pd

#### Parameters  

In [53]:
csv_in = 'dm-end1-3.csv'
# min number of common items between target user's evaluation and items in DB
min_common_items = 3

# To show more rows and columns
pd.options.display.max_rows = 999 
pd.options.display.max_columns = 999 

#### Read CSV file  

In [54]:
df_orig = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
df_orig = df_orig.replace(-1, np.nan)
#df.index = df.columns
print(df_orig.shape)
print(df_orig.info())
display(df_orig.head())

(50, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       23 non-null     float64
 1   B       24 non-null     float64
 2   C       23 non-null     float64
 3   D       28 non-null     float64
 4   E       28 non-null     float64
 5   F       30 non-null     float64
 6   G       25 non-null     float64
 7   H       25 non-null     float64
 8   I       30 non-null     float64
 9   J       29 non-null     float64
dtypes: float64(10)
memory usage: 4.0 KB
None


,A,B,C,D,E,F,G,H,I,J
0,4.0,NaN,NaN,4.0,NaN,NaN,NaN,1.0,0.0,NaN
1,NaN,NaN,NaN,3.0,4.0,0.0,1.0,NaN,2.0,3.0
2,3.0,2.0,0.0,NaN,2.0,4.0,1.0,NaN,NaN,NaN
3,NaN,NaN,0.0,NaN,1.0,1.0,NaN,0.0,NaN,4.0
4,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,1.0,NaN


相関行列を計算し、dfに格納   
compute correlation matrix and store it to the DataFrame df


In [55]:
df = df_orig.corr()
print(df.shape)
print(df)

(10, 10)
          A         B         C         D         E         F         G  \
A  1.000000  0.000000 -0.346348  0.124781 -0.127381  0.315885  0.253388   
B  0.000000  1.000000 -0.063112  0.103448 -0.297775  0.205456 -0.319334   
C -0.346348 -0.063112  1.000000 -0.477567  0.376275 -0.342410 -0.050043   
D  0.124781  0.103448 -0.477567  1.000000 -0.102489  0.062943 -0.023618   
E -0.127381 -0.297775  0.376275 -0.102489  1.000000 -0.348724 -0.316176   
F  0.315885  0.205456 -0.342410  0.062943 -0.348724  1.000000  0.325913   
G  0.253388 -0.319334 -0.050043 -0.023618 -0.316176  0.325913  1.000000   
H -0.113151 -0.017153 -0.159901  0.042618  0.295820  0.053822  0.191237   
I -0.230590  0.220950 -0.436306  0.159281 -0.380018 -0.026504 -0.172030   
J  0.521481  0.441975 -0.593291  0.364450  0.011155 -0.031490  0.409010   

          H         I         J  
A -0.113151 -0.230590  0.521481  
B -0.017153  0.220950  0.441975  
C -0.159901 -0.436306 -0.593291  
D  0.042618  0.159281  0.3644

In [56]:
def predict_scores(df_sim, ser_target):
    ret = {}
    for item1 in df_sim.index:  # not yet rated by the target user
        v1 = df_sim.loc[item1]
        #if v1.isnull().sum() > 0:  # debug
        #    print('v1:',v1)  # debug
        if v1.notnull().sum() < min_common_items: continue
        v11 = v1[ v1.notnull() ]
        t11 = ser_target[ v1.notnull() ]
        pred1 = (v11 * t11).sum() / np.abs(v11).sum()
        #print('v11:',v11)  # debug
        #print('t11:',t11)  # debug
        #print('pred1:',pred1)  # debug
        ret[item1] = pred1
    
    ser_ret = pd.Series(ret)
    
    return ser_ret.sort_values(ascending=False)

Function for user-based collaborative filtering.  

arguments: dictionary of scores for the target user  
and the number of items to recommend.  

ex)
```
get_recomm_by_user_sim(df, {'maguro':1, 'ika':1, 'uni':3,
                        'awabi':4, 'hirame':4, 'aoyagi':4})  
```
-> return list such as [('akagai', 2.9835603009918303), ('mirugai', 2.945676429588114), ...]

In [57]:
def get_recomm_by_item_sim(df, target_dic):
    ser_target = pd.Series(target_dic)
    #print(target_dic)  # debug
    #print(ser_target)  # debug
    # make dataframe with columns included in target_dic
    #print(df.shape)  # debug
    df_scores = df[ ser_target.index ]
    #print(df_scores.shape)  # debug
    # drop rows included in target_dic (already rated)
    df_scores = df_scores.drop(index=ser_target.index)
    #print(df_scores.shape)  # debug
    #display(df_scores.head())  # debug
    recomm = predict_scores(df_scores, ser_target)
    
    return recomm

#### Do recommendation  

In [58]:
recomm = get_recomm_by_item_sim(df,
                                {'A':0, 'B':0, 'C':4, 'D': 4})
print('Number of items calculated:', len(recomm))
print('Recommendation:')
print(recomm.head())

Number of items calculated: 6
Recommendation:
E    1.211548
G   -0.455837
J   -0.476453
I   -1.058230
F   -1.206297
dtype: float64
